In [2]:
import json
import pandas as pd
from collections import defaultdict
import os
from collections import Counter

In [3]:
pip install langchain langchain_community transformers SQLAlchemy==2.0.27 openai faiss-cpu pinecone-client

Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain.docstore.document import Document

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain_community.llms.bedrock import Bedrock
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import pinecone 
from langchain.vectorstores import Pinecone
from pinecone import Pinecone, ServerlessSpec
from langchain.embeddings import BedrockEmbeddings
from langchain.chains import RetrievalQA

In [6]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

In [7]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
ls

Question_1.ipynb               TRDataChallenge2023.txt
Question_2_copy.ipynb          loading+data+and+summarizing-coding.ipynb
Question_2_copy_new.ipynb      my_list.json
Question_2_first_method.ipynb  utils/


In [9]:
#Read The Data

In [10]:
with open('TRDataChallenge2023.txt', 'r') as file:
    json_objects = []
    for line in file:
        json_data = json.loads(line)
        json_objects.append(json_data)

In [11]:
len(json_objects)

18000

In [12]:
empty_postures=0

In [13]:
for i in json_objects:
    if len(i['postures'])==0:
        empty_postures+=1

In [14]:
empty_postures

923

In [15]:
for i in json_objects:
    if len(i['postures'])==0:
        json_objects.remove(i)

In [16]:
len(json_objects)

17129

In [17]:
empty_postures=0

In [18]:
for i in json_objects:
    if len(i['postures'])==0:
        json_objects.remove(i)

In [19]:
len(json_objects)

17077

In [20]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

Create new client
  Using region: us-west-2
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)


In [21]:
inference_modifier = {
    "temperature": 0.9,
    "p": 0.9,
    "max_tokens": 4096
}

textgen_llm= Bedrock(
    model_id="cohere.command-text-v14",
    client=boto3_bedrock,
    model_kwargs=inference_modifier,
)

In [22]:
summary_chain = load_summarize_chain(
    llm=textgen_llm,
    chain_type='map_reduce'
)

In [23]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0,
)

In [24]:
#Second Method - RAG from Raw Documents 

In [25]:
#Due to Time Contraint I decided to take top 10 Postures by count. Because LLM is taking for ever 
#To run command. I will take 100 Documentsfrom top 10 that means 10 from each


In [26]:
#taking top 10

In [27]:
postures_unique=[i['postures'] for i in json_objects]

In [28]:
posture_list=[]
for i in postures_unique:
    posture_list.extend(i)

In [29]:
posture_list_final = list(dict(Counter(posture_list)).keys())[:10]

In [30]:
posture_list_final

['On Appeal',
 'Appellate Review',
 'Sentencing or Penalty Phase Motion or Objection',
 'Motion to Compel Arbitration',
 'Review of Administrative Decision',
 'Motion to Dismiss',
 'Trial or Guilt Phase Motion or Objection',
 'Motion for Preliminary Injunction',
 'Motion for Judgment as a Matter of Law (JMOL)/Directed Verdict',
 "Motion for Attorney's Fees"]

In [31]:
#taking 100 documents from top 10

In [32]:
json_objects_new=[]
for i in json_objects:
    for j in i['postures']:
        if j in posture_list_final:
            json_objects_new.append(i)

In [33]:
final_list=[]
doc_id=[-1]
for i in posture_list_final:
    temp_list=[]
    for j in json_objects_new:
        if i in j['postures']:
            if j['documentId'] not in doc_id:
                doc_id.append(j['documentId'])
                temp_list.append(j)
            else:
                pass
        if len(temp_list)==10:
            final_list.extend(temp_list)
            break

In [34]:
for i in final_list:
    for j in i['postures']:
        if j not in posture_list_final:
            i['postures'].remove(j)

In [35]:
for i in final_list:
    print(i['postures'])

['On Appeal']
['Motion to Compel Arbitration', 'On Appeal']
['On Appeal', 'Review of Administrative Decision']
['On Appeal']
['On Appeal']
['On Appeal']
['On Appeal']
['On Appeal']
['On Appeal', 'Review of Administrative Decision']
['On Appeal']
['Appellate Review', 'Sentencing or Penalty Phase Motion or Objection']
['Appellate Review', 'Sentencing or Penalty Phase Motion or Objection']
['Appellate Review', 'Trial or Guilt Phase Motion or Objection']
['Appellate Review']
['Appellate Review']
['Appellate Review']
['Appellate Review']
['Appellate Review']
['Appellate Review']
['Appellate Review']
['Appellate Review', 'Sentencing or Penalty Phase Motion or Objection']
['Appellate Review', 'Sentencing or Penalty Phase Motion or Objection']
['Appellate Review', 'Sentencing or Penalty Phase Motion or Objection']
['Appellate Review', 'Sentencing or Penalty Phase Motion or Objection']
['Appellate Review', 'Sentencing or Penalty Phase Motion or Objection']
['Appellate Review', 'Sentencing or Pe

In [36]:
for i in final_list:
    txt=''
    for j in i['sections']:
        if len(j['paragraphs'])==0:
            txt=txt+''
        else:
            for k in range(len(j['paragraphs'])):
                txt=txt+j['paragraphs'][k]
    i['para_concat']=txt

In [37]:
keys_to_keep = ["postures", "para_concat"]

In [38]:
new_data = [{key: item[key] for key in keys_to_keep} for item in final_list]

In [39]:
df=pd.DataFrame(new_data,columns=['postures','para_concat'])

In [40]:
df=df.explode('postures')

In [41]:
aggregated_text_by_posture = df.groupby('postures')['para_concat'].apply(lambda x:''.join(x)).reset_index()

In [42]:
aggregated_text_by_posture

,postures,para_concat
0,Appellate Review,"After pleading guilty, William Jerome Howard, ..."
1,Motion for Attorney's Fees,Gregory Kinsey (“Kinsey”) appeals from the tri...
2,Motion for Contempt Sanctions,"PlayNation Play Systems, Inc., a company that ..."
3,Motion for Judgment as a Matter of Law (JMOL)/...,The pending proceeding was brought pursuant to...
4,Motion for Preliminary Injunction,The opinion of the court was delivered byIn th...
5,Motion for Remittitur,Pending are Defendant International Longshorem...
6,Motion to Compel Arbitration,"Frederick Greene, the plaintiff below, derivat..."
7,Motion to Dismiss,The opinion of the court was delivered byIn th...
8,Motion to Stay Enforcement of Judgment,A borough resident filed suit against the boro...
9,Motion to Transfer or Change Venue,These motions arise out of complicated lending...


In [43]:
data_dict={}

In [44]:
for index,row in df.iterrows():
    data_dict[row['postures']]=row['para_concat']

In [45]:
for i in data_dict:
    print(i)

On Appeal
Motion to Compel Arbitration
Review of Administrative Decision
Appellate Review
Sentencing or Penalty Phase Motion or Objection
Trial or Guilt Phase Motion or Objection
Motion for Preliminary Injunction
Motion to Dismiss
Motion to Transfer or Change Venue
Motion for Attorney's Fees
Motion for Judgment as a Matter of Law (JMOL)/Directed Verdict
Motion for Remittitur
Motion to Stay Enforcement of Judgment
Motion for Contempt Sanctions


In [46]:
data_dict_Doc={}

In [47]:
data_dict_Doc

{}

In [48]:
for i in data_dict:
    data_dict_Doc[i]=text_splitter.split_text(data_dict[i])

In [49]:
docs=[]
for i in data_dict_Doc:
    for j in data_dict_Doc[i]:
        docs.append(Document(page_content=j,metadata={'postures': i,}))

In [50]:
len(docs)

348

In [51]:
bedrock_embeddings = BedrockEmbeddings(model_id="cohere.embed-english-v3", client=boto3_bedrock)

In [52]:
index = FAISS.from_documents(docs, bedrock_embeddings)

In [53]:
retriever = index.as_retriever()
retriever.search_kwargs['fetch_k'] = 20
retriever.search_kwargs['maximal_marginal_relevance'] = True
retriever.search_kwargs['k'] = 10

In [54]:
chain_search = RetrievalQA.from_chain_type(
    llm=textgen_llm, 
    retriever=retriever,
    chain_type='map_reduce',
    verbose=False,
    return_source_documents=True)

In [55]:
#Testing 

In [56]:
#taking another 30 documents 

In [57]:
test_list=[]
#doc_id=[-1]
for i in posture_list_final:
    temp_list=[]
    for j in json_objects_new:
        if i in j['postures']:
            if j['documentId'] not in doc_id:
                doc_id.append(j['documentId'])
                temp_list.append(j)
            else:
                pass
        if len(temp_list)==3:
            test_list.extend(temp_list)
            break

In [58]:
for i in test_list:
    for j in i['postures']:
        if j not in posture_list_final:
            i['postures'].remove(j)

In [59]:
for i in test_list:
    print(i['postures'])

['On Appeal']
['On Appeal']
['On Appeal']
['Appellate Review']
['Appellate Review']
['Appellate Review']
['Appellate Review', 'Sentencing or Penalty Phase Motion or Objection']
['Appellate Review', 'Sentencing or Penalty Phase Motion or Objection']
['Appellate Review', 'Sentencing or Penalty Phase Motion or Objection']
['Motion to Compel Arbitration', 'On Appeal']
['Motion to Compel Arbitration']
['Motion to Compel Arbitration', 'Motion to Dismiss', 'On Appeal']
['On Appeal', 'Review of Administrative Decision']
['Review of Administrative Decision']
['Review of Administrative Decision']
['Motion to Dismiss']
['Motion to Dismiss', 'On Appeal']
['Motion to Dismiss', 'On Appeal']
['Appellate Review', 'Trial or Guilt Phase Motion or Objection']
['Appellate Review', 'Trial or Guilt Phase Motion or Objection']
['Appellate Review', 'Trial or Guilt Phase Motion or Objection']
['Motion for Preliminary Injunction', 'On Appeal']
['Motion for Preliminary Injunction', 'On Appeal']
['Motion for Prel

In [60]:
for i in test_list:
    txt=''
    for j in i['sections']:
        if len(j['paragraphs'])==0:
            txt=txt+''
        else:
            for k in range(len(j['paragraphs'])):
                txt=txt+j['paragraphs'][k]
    i['para_concat']=txt

In [61]:
test_list[0].keys()

dict_keys(['documentId', 'postures', 'sections', 'para_concat'])

In [62]:
keys_to_keep = ["postures", "para_concat"]

In [63]:
new_data = [{key: item[key] for key in keys_to_keep} for item in test_list]

In [64]:
len(new_data)

30

In [65]:
df=pd.DataFrame(new_data,columns=['postures','para_concat'])

In [66]:
df

,postures,para_concat
0,[On Appeal],This appeal arises from a judgment entered in ...
1,[On Appeal],"The plaintiff, Gaynell Shelton, appeals a tria..."
2,[On Appeal],Appeal from an order of the District Court of ...
3,[Appellate Review],Defendants-appellants Kwame A. Insaidoo (“Kwam...
4,[Appellate Review],¶ 1. Defendant appeals a conviction of refusal...
5,[Appellate Review],¶1. A jury found Willie Nash guilty of possess...
6,"[Appellate Review, Sentencing or Penalty Phase...",Defendant appeals a judgment of conviction for...
7,"[Appellate Review, Sentencing or Penalty Phase...",A jury found Jorge Sainz Navarrete guilty of c...
8,"[Appellate Review, Sentencing or Penalty Phase...",Elmer Don Whittaker pled guilty to maintaining...
9,"[Motion to Compel Arbitration, On Appeal]","Appellant Brickell Motors, LLC, seeks review o..."


In [259]:
#df=df.explode('postures').reset_index(drop=True)

In [260]:
#df

In [306]:
test_summaries=[]

In [307]:
for i in list(df.para_concat):
    test_summaries.append(summary_chain.invoke(text_splitter.create_documents([i])))

In [308]:
test_summaries_final=[]
for i in test_summaries:
    test_summaries_final.append(i['output_text'])

In [311]:
with open('my_list.json', 'w') as f:
    json.dump(test_summaries_final, f)

In [67]:
with open('my_list.json', 'r') as f:
    restored_list = json.load(f)

In [68]:
len(restored_list)

30

In [73]:
final_test_output_method_2=[]

In [74]:
for i in restored_list:
    temp_list=[]
    try:
        for j in index.similarity_search_with_relevance_scores(i):
            temp_list.append(j[0].metadata['postures'])
        temp_list=list(set(temp_list))
        final_test_output_method_2.append(temp_list)
    except:
        temp_list=['LLM Error']
        final_test_output_method_2.append(temp_list)

In [75]:
final_test_output_method_2

[['Motion for Judgment as a Matter of Law (JMOL)/Directed Verdict',
  'Motion for Remittitur'],
 ['On Appeal', 'Motion for Contempt Sanctions', "Motion for Attorney's Fees"],
 ['Motion to Dismiss',
  'Review of Administrative Decision',
  'Motion for Preliminary Injunction'],
 ['Appellate Review',
  'Sentencing or Penalty Phase Motion or Objection',
  'Motion to Transfer or Change Venue',
  'Trial or Guilt Phase Motion or Objection'],
 ['Appellate Review', 'Trial or Guilt Phase Motion or Objection'],
 ['Sentencing or Penalty Phase Motion or Objection'],
 ['Sentencing or Penalty Phase Motion or Objection',
  'Trial or Guilt Phase Motion or Objection'],
 ['LLM Error'],
 ['Sentencing or Penalty Phase Motion or Objection',
  'On Appeal',
  'Motion for Contempt Sanctions'],
 ['Motion to Transfer or Change Venue', 'Motion to Compel Arbitration'],
 ['Motion to Transfer or Change Venue', 'Motion to Compel Arbitration'],
 ['Motion to Transfer or Change Venue', 'Motion to Compel Arbitration'],
 

In [76]:
True_Posture_List_Test=list(df.postures)

In [77]:
with open('True_Posture_List_Test.json', 'w') as f:
    json.dump(True_Posture_List_Test, f)

In [78]:
with open('True_Posture_List_Test.json', 'r') as f:
    true_list = json.load(f)

In [79]:
final_prediction_method_2=pd.DataFrame({'True_Postures':true_list, 'Predicted_Postures':final_test_output_method_2})

In [80]:
final_prediction_method_2.to_csv('Prediction_Method_2.csv')